#### This assignment may be worked individually or in pairs. Enter your name/names here:
    

In [44]:
#names here

# Assignment 1: Decision Trees

In this assignment we'll implement the Decision Tree algorithm to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains features from the Messidor image set to predict whether an image contains signs of diabetic retinopathy or not. This dataset has `1151` instances and `20` attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

Attribute Information:

0) The binary result of quality assessment. 0 = bad quality 1 = sufficient quality.

1) The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack. 

2-7) The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5, . . . , 1, respectively. 

8-15) contain the same information as 2-7) for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the 
number of lesions with the diameter of the ROI to compensate different image sizes. 

16) The euclidean distance of the center of the macula and the center of the optic disc to provide important information regarding the patient's condition. This feature is also normalized with the diameter of the ROI.

17) The diameter of the optic disc. 

18) The binary result of the AM/FM-based classification.

19) Class label. 1 = contains signs of Diabetic Retinopathy (Accumulative label for the Messidor classes 1, 2, 3), 0 = no signs of Diabetic Retinopathy.

#### Implementation: 
A few function prototypes are already given to you, please don't change those. You can add additional helper functions for your convenience. 

*Suggestion:* The dataset is substantially big, for the purpose of easy debugging, work with a subset of the data and test your decision tree implementation on that.

#### Notes:
Parts of this assignment will be **autograded** so a couple of caveats :-
- Entropy is calculated using log with base 2, `math.log2(x)`.
- For continuous features ensure that the threshold value lies exactly between 2 buckets. For example, if for feature 2 the best split occurs between 10 and 15 then the threshold value will be set as 12.5.
- For binary features [0/1] the threshold value will be 0.5. All values < `thresh_val` go to the left child and all values >= `thresh_val` go to the right child.

In [45]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import log2
from random import shuffle

In [46]:
class DataPoint:
    def __str__(self):
        return "< " + str(self.label) + ": " + str(self.features) + " >"
    def __init__(self, label, features):
        self.label = label # the classification label of this data point
        self.features = features

Q1. Read data from a CSV file. Put it into a list of `DataPoints`.

In [47]:
def get_data(filename):
    data = []
    file = pd.read_csv(filename, header=None).to_numpy()

    for dp in file:
        label = dp[19]
        features = dp[:19]
        data.append(DataPoint(label, features))
    
    return data

In [48]:
class TreeNode:
    is_leaf = True          # boolean variable to check if the node is a leaf
    feature_idx = None      # index that identifies the feature
    thresh_val = None       # threshold value that splits the node
    prediction = None       # prediction class (only valid for leaf nodes)
    left_child = None       # left TreeNode (all values < thresh_val)
    right_child = None      # right TreeNode (all values >= thresh_val)
    
    def printTree(self):    # for debugging purposes
        if self.is_leaf:
            print ('Leaf Node:      predicts ' + str(self.prediction))
        else:
            print ('Internal Node:  splits on feature ' 
                   + str(self.feature_idx) + ' with threshold ' + str(self.thresh_val))
            self.left_child.printTree()
            self.right_child.printTree()

Q2. Implement the function `make_prediction` that takes the decision tree root and a `DataPoint` instance and returns the prediction label.

In [49]:
def make_prediction(tree_root, data_point):
#     your code goes here
    while not tree_root.is_leaf:
        if data_point.features[tree_root.feature_idx] < tree_root.thresh_val:
            tree_root = tree_root.left_child
        else:
            tree_root = tree_root.right_child

    return tree_root.prediction

Q3. Implement the function `split_dataset` given an input data set, a `feature_idx` and the `threshold` for the feature. `left_split` will have all values < `threshold` and `right_split` will have all values >= `threshold`.

In [50]:
def split_dataset(data, feature_idx, threshold):
    left_split = []
    right_split = []
#     your code goes here
    for dp in data:
        if dp.features[feature_idx] < threshold:
            left_split.append(dp)
        else:
            right_split.append(dp)

    return (left_split, right_split)

Q4. Implement the function `calc_entropy` to return the entropy of the input dataset.

In [51]:
from scipy.stats import entropy

def calc_entropy(data):
#     entropy = 0.0
    label_count = {}

    for dp in data:
        if dp.label in label_count:
            label_count[dp.label] += 1
        else:
            label_count[dp.label] = 1
            
    p_list = []
    total = sum(label_count.values())
    
    for label in label_count:
        p_list.append(label_count[label] / total)
    
    return entropy(p_list, base=2)

# def calc_entropy(data):
#     entropy = 0.0
# #     your code goes here
#     num_label_0 = 0
#     num_label_1 = 0
    
#     if len(data) == 0:
#         return entropy
    
#     for data_point in data:
#         if data_point.label == 0:
#             num_label_0 += 1
#         else:
#             num_label_1 += 1

#     prob_0 = num_label_0 / len(data)
#     prob_1 = num_label_1 / len(data)
    
#     # Check if either prob_0 or prob_1 is 0
#     if prob_0 > 0 and prob_1 == 0:
#         entropy = - prob_0 * log2(prob_0)
#     elif prob_0 == 0 and prob_1 > 0:
#         entropy = - prob_1 * log2(prob_1)
#     else:
#         entropy = - prob_0 * log2(prob_0) - prob_1 * log2(prob_1)
        
#     return entropy

Q5. Implement the function `calc_best_threshold` which returns the best information gain and the corresponding threshold value for one feature at `feature_idx`.

In [52]:
def calc_best_threshold(data, feature_idx):
    best_info_gain = 0.0
    best_thresh = None
#     your code goes here
    sorted_data = sorted(data, key=lambda dp: dp.features[feature_idx])
    entropy = calc_entropy(sorted_data)
    lowest = float('inf')
    
    for i in range(len(sorted_data) - 1):
        feature_check = sorted_data[i].features[feature_idx] != sorted_data[i + 1].features[feature_idx]
        label_check = sorted_data[i].label != sorted_data[i + 1].label
        threshold = (sorted_data[i].features[feature_idx] + sorted_data[i + 1].features[feature_idx]) / 2
        
        if feature_check and label_check:
            left = sorted_data[:i + 1]
            right = sorted_data[i + 1:]
            entropy_left = (len(left) / len(sorted_data)) * calc_entropy(left)
            entropy_right = (len(right) / len(sorted_data)) * calc_entropy(right)
            entropy_split = entropy_left + entropy_right
            
            if entropy_split < lowest:
                lowest = entropy_split
                best_thresh = threshold
                
    best_info_gain = entropy - lowest
    
    return (best_info_gain, best_thresh)

Q6. Implement the function `identify_best_split` which returns the best feature to split on for an input dataset, and also returns the corresponding threshold value.

In [53]:
def identify_best_split(data):
    if len(data) < 2:
        return (None, None)
    best_feature = None
    best_thresh = None
#     your code goes here
    best_info_gain = 0
    
    for i in range(len(data[0].features)):
        cur_info_gain, cur_thresh = calc_best_threshold(data, i)
        
        if cur_info_gain > best_info_gain:
            best_info_gain = cur_info_gain
            best_feature = i
            best_thresh = cur_thresh
    
    return (best_feature, best_thresh)

Q7. Implement the function `create_leaf_node` which returns a `TreeNode` with `is_leaf=True` and `prediction` set to whichever classification occurs most in the dataset at this node.

In [54]:
def create_leaf_node(data):
#     your code goes here
    node = TreeNode()
    node.is_leaf = True
    node.prediction = 0
    num_0 = 0
    num_1 = 0
    
    for dp in data:
        if dp.label == 0:
            num_0 += 1
        else:
            num_1 += 1
            
    if num_1 > num_0:
        node.prediction = 1
        
    return node

Q8. Implement the `create_decision_tree` function. `max_levels` denotes the maximum height of the tree (for example if `max_levels = 1` then the decision tree will only contain the leaf node at the root. [Hint: this is where the recursion happens.]

In [55]:
def create_decision_tree(data, max_levels):
#     your code goes here
    if max_levels == 1:
        return create_leaf_node(data)
    else:
        best_feature, best_thresh = identify_best_split(data)
        
        if best_feature == None and best_thresh == None:
            return create_leaf_node(data)
        
        left_split, right_split = split_dataset(data, best_feature, best_thresh)
        
        node = TreeNode()
        node.is_leaf = False
        node.feature_idx = best_feature
        node.thresh_val = best_thresh
        node.left_child = create_decision_tree(left_split, max_levels - 1)
        node.right_child = create_decision_tree(right_split, max_levels - 1)
        
        return node

Q9. Given a test set, the function `calc_accuracy` returns the accuracy of the classifier. You'll use the `make_prediction` function for this.

In [56]:
def calc_accuracy(tree_root, data):
#     your code goes here
    correct = 0
    
    for dp in data:
        pred = make_prediction(tree_root, dp)
        actual = dp.label
        
        if pred == actual:
            correct += 1
            
    return correct / len(data)

Q10. Keeping the `max_levels` parameter as 10, use 5-fold cross validation to measure the accuracy of the model. Print the accuracy of the model.

In [57]:
# edit the code here - this is just a sample to get you started
import time

d = get_data("messidor_features.txt")
max_levels = 10
k = 5
total_accuracy = 0.0

for i in range(k):
    # partition data into train_set and test_set
    test_start = int((i / k) * len(d))
    test_end = int(((i + 1) / k) * len(d))
    train_set = d[:test_start] + d[test_end:]
    test_set = d[test_start:test_end]

    print ('Training set size:', len(train_set))
    print ('Test set size    :', len(test_set))

    # create the decision tree
    start = time.time()
    tree = create_decision_tree(train_set, 10)
    end = time.time()
    print ('Time taken:', end - start)

    # calculate the accuracy of the tree
    accuracy = calc_accuracy(tree, test_set)
    total_accuracy += accuracy
    print ('The accuracy on the test set is ', str(accuracy * 100.0))
    
print("\nTotal Accuracy:", total_accuracy / k * 100)
# t.printTree()

Training set size: 921
Test set size    : 230
Time taken: 4.214568138122559
The accuracy on the test set is  63.47826086956522
Training set size: 921
Test set size    : 230
Time taken: 3.8990609645843506
The accuracy on the test set is  64.34782608695652
Training set size: 921
Test set size    : 230
Time taken: 4.160631895065308
The accuracy on the test set is  68.69565217391305
Training set size: 921
Test set size    : 230
Time taken: 3.932494878768921
The accuracy on the test set is  64.34782608695652
Training set size: 920
Test set size    : 231
Time taken: 3.7363760471343994
The accuracy on the test set is  64.93506493506493

Total Accuracy: 65.16092603049125
